In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Unzip here
!unzip "/content/drive/MyDrive/bert.zip"

Archive:  /content/drive/MyDrive/bert.zip
   creating: new_saved_model/bert/
   creating: new_saved_model/bert/assets/
  inflating: new_saved_model/bert/assets/vocab.txt  
   creating: new_saved_model/bert/variables/
  inflating: new_saved_model/bert/variables/variables.data-00000-of-00001  
  inflating: new_saved_model/bert/variables/variables.index  
  inflating: new_saved_model/bert/saved_model.pb  


In [3]:
!ls

drive  new_saved_model	sample_data


In [25]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official import nlp
reloaded = tf.saved_model.load('new_saved_model/bert')

In [12]:
 model = tf.keras.models.load_model('new_saved_model/bert',compile=False)

In [18]:
!mkdir text
!mkdir text/clickbait
!mkdir text/normal

In [30]:
githubUrl= 'https://raw.githubusercontent.com/mtp9k/Identifying-Clickbait-SYS-60616-/main/'
full = f'{githubUrl}clickbait_data.csv'
DATA_ENDPOINT = full

In [32]:
import pandas as pd
df = pd.read_csv(DATA_ENDPOINT)
df.to_csv('clickbait_train.csv',index=False)
df = pd.read_csv('clickbait_train.csv')
normal = df.loc[df['clickbait']==0]['headline'].values
clickbait = df.loc[df['clickbait']==1]['headline'].values
def generateTensorflowTextDir(data,folderName):
    for i,line in enumerate(data):
        filename = f'{i}_{folderName}'
        with open(f"text/{folderName}/{filename}.txt", "w") as outfile:
            outfile.write(line)
generateTensorflowTextDir(normal,'normal')
generateTensorflowTextDir(clickbait,'clickbait')
# Batch Size -- we set to 1 to just read it in
BATCH_SIZE = 64
DATASET_SIZE = 32000
train_size = 32000 - 6400
test_size =6400
seed = 49
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'text',
    batch_size=BATCH_SIZE,
    class_names=['normal','clickbait'],
    subset='training',
    validation_split=0.2, 
    seed=seed
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
   'text/',
    batch_size=BATCH_SIZE,
    class_names=['normal','clickbait'],
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 32000 files belonging to 2 classes.
Using 25600 files for training.
Found 32000 files belonging to 2 classes.
Using 6400 files for validation.


In [27]:
from tensorflow.keras import losses
import tensorflow as tf
from official import nlp
import official.nlp.optimization
loss=losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.1)
epochs=3
metrics = tf.metrics.BinaryAccuracy()
steps_per_epoch = tf.data.experimental.cardinality(raw_train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

model.compile(optimizer=optimizer,
                        loss=loss,
                        metrics=metrics)

In [ ]:
history = model.fit(
    raw_train_ds,
    validation_data=raw_val_ds,
    epochs=epochs)

In [ ]:
export_dir='new_saved_model/bert'
tf.saved_model.save(best_bert_model, export_dir=export_dir)


In [ ]:
!zip -r new_saved_model/bert.zip new_saved_model/bert

In [ ]:
from google.colab import files
files.download("new_saved_model/bert.zip")